In [ ]:
import pandas as pd
import json
import xgboost as xgb
import pickle
import lightgbm as lgb
import numpy as np
from numpy import mean
from statsforecast.models import (
    HistoricAverage,
    Naive,
    RandomWalkWithDrift,
    SeasonalNaive,
    WindowAverage,
    SeasonalWindowAverage,
    ADIDA,
    CrostonClassic,
    CrostonSBA,
    IMAPA,
    TSB,
    Theta,
    AutoARIMA,
    OptimizedTheta,
    AutoCES,
    AutoETS,
    DynamicTheta,
    SimpleExponentialSmoothing,
    SimpleExponentialSmoothingOptimized,
    SeasonalExponentialSmoothing,
    SeasonalExponentialSmoothingOptimized,
    Holt,
    HoltWinters,
)

baseline_model_dict = {
    "historic_average_baseline": HistoricAverage,
    "naive_baseline": Naive,
    "random_walk_baseline": RandomWalkWithDrift,
    # "window_average_baseline": WindowAverage,
}

seasonal_baseline_model_dict = {
    "seasonal_naive_baseline": SeasonalNaive,
    # "seasonal_window_average_baseline": SeasonalWindowAverage,
}

model_dict = {
    # 'adida_model':ADIDA,
    # 'croston_classic_model':CrostonClassic,
    # 'croston_sba_model':CrostonSBA,
    # 'imapa_model':IMAPA,
    # 'tsb_model':TSB,
    'theta_model': Theta,
    # 'auto_arima_model':AutoARIMA,
    'optimized_theta_model': OptimizedTheta,
    'auto_ces_model': AutoCES,
    'auto_ets_model': AutoETS,
    'dynamic_theta_model': DynamicTheta,
    # 'simple_exponential_smoothing_model':SimpleExponentialSmoothing,
    'simple_exponential_smoothing_optimized_model': SimpleExponentialSmoothingOptimized,
    # 'seasonal_exponential_smoothing_model':SeasonalExponentialSmoothing,
    'seasonal_exponential_smoothing_optimized_model': SeasonalExponentialSmoothingOptimized,
    'holt_model': Holt,
    'holt_winters_model': HoltWinters,
}

raw_sales_data = pd.read_csv(
    'gs://gfk-eco-local-forecast/simulations/neo_backtest_regular/Weekly/2780/backtests/raw_sales_data.csv')

all_cell_rows = ['-'.join(value) for value in raw_sales_data[['country_code', 'item_group_code', 'outlet_group_code']].values]

raw_sales_data['cell'] = all_cell_rows

def get_metrics(prediction_dataframe, prediction_column, actual_column):

    e = prediction_dataframe[prediction_column].values - \
        prediction_dataframe[actual_column].values

    ae, se, pe = abs(e), e**2, (e/prediction_dataframe[actual_column].values)

    mae, mse, mpe, ape = mean(ae), mean(se), mean(
        pe), (ae/prediction_dataframe[actual_column].values)

    rmse, mape = mse**0.5, mean(ape)

    return {'mae': mae, 'mse': mse, 'mpe': mpe, 'rmse': rmse, 'mape': mape}


def get_model_predictions(model_dict, univariant_data, train_data, actual_values, forecast_horizon, seasonal):

    model_result = {}
    model_predictions = {}
    model_inpredictions = {}

    for model_name, model in model_dict.items():
        try:
            print(model_name)

            try:
                intmodel = model(season_length=seasonal)
            except:
                intmodel = model()

            fitted_model = intmodel.forecast(
                y=train_data, fitted=True, h=forecast_horizon)

            mean_prediction = fitted_model['mean']

            insample_prediction = fitted_model['fitted']
            insample_prediction[np.isnan(insample_prediction)] = 0

            prediction_dataframe = pd.DataFrame(
                mean_prediction, actual_values).reset_index()
            prediction_dataframe.columns = ["actual", "prediction"]

            model_result[model_name] = get_metrics(
                prediction_dataframe, 'prediction', 'actual')
            model_predictions[model_name] = mean_prediction
            model_inpredictions[model_name] = insample_prediction

        except Exception as e:
            print(e)
            print(model_predictions)

    model_predictions_df = pd.DataFrame(
        model_predictions, index=univariant_data.index[-forecast_horizon:]).fillna(0)
    model_result_df = pd.DataFrame(model_result).fillna(0)
    model_inpredictions_df = pd.DataFrame(
        model_inpredictions, univariant_data.index[:-forecast_horizon]).fillna(0)

    return model_predictions_df, model_inpredictions_df, model_result_df


def get_baseline_forecasts(baseline_model_dict, seasonal_baseline_model_dict, model_dict, univariant_data, train_data, actual_values, forecast_horizon, seasonal, window):

    baseline_result = {}
    baseline_predictions = {}
    baseline_inpredictions = {}

    for model_name, model in baseline_model_dict.items():

        print(model_name)

        if 'window' in model_name:
            intmodel = model(window_size=window)

        else:
            intmodel = model()

        fitted_model = intmodel.forecast(
            y=train_data, fitted=True, h=forecast_horizon)

        mean_prediction = fitted_model['mean']
        insample_prediction = fitted_model['fitted']

        prediction_dataframe = pd.DataFrame(
            mean_prediction, actual_values).reset_index()
        prediction_dataframe.columns = ["actual", "prediction"]

        baseline_result[model_name] = get_metrics(
            prediction_dataframe, 'prediction', 'actual')

        baseline_predictions[model_name] = mean_prediction
        baseline_inpredictions[model_name] = insample_prediction

    for model_name, model in seasonal_baseline_model_dict.items():
        print(model_name)

        if 'window' in model_name:
            intmodel = model(season_length=seasonal, window_size=1)

        else:
            intmodel = model(season_length=seasonal)

        fitted_model = intmodel.forecast(
            y=train_data, fitted=True, h=forecast_horizon)

        mean_prediction = fitted_model['mean']

        insample_prediction = fitted_model['fitted']
        insample_prediction[np.isnan(insample_prediction)] = 0

        prediction_dataframe = pd.DataFrame(
            mean_prediction, actual_values).reset_index()
        prediction_dataframe.columns = ["actual", "prediction"]

        baseline_result[model_name] = get_metrics(
            prediction_dataframe, 'prediction', 'actual')
        baseline_predictions[model_name] = mean_prediction
        baseline_inpredictions[model_name] = insample_prediction

    baseline_predictions['actual'] = actual_values
    baseline_inpredictions['actual'] = train_data

    baseline_predictions_df = pd.DataFrame(
        baseline_predictions, index=univariant_data.index[-forecast_horizon:]).fillna(0)
    baseline_result_df = pd.DataFrame(baseline_result).fillna(0)
    baseline_inpredictions_df = pd.DataFrame(
        baseline_inpredictions, univariant_data.index[:-forecast_horizon]).fillna(0)
    
    return baseline_predictions_df, baseline_inpredictions_df, baseline_result_df


In [2]:
final_results = {}
for cell in set(all_cell_rows):

    full_data = raw_sales_data[raw_sales_data['cell']==cell].sort_values(by='period_seq').set_index("start_date")

    full_data.index = pd.to_datetime(full_data.index, format="%Y-%m-%d")

    single_column = "quantity"

    univariant_data = full_data[[single_column]]

    if univariant_data.shape[0] < 149:
        continue

    univariant_data = univariant_data
    univariant_data[single_column] = univariant_data[single_column].astype(float)

    forecast_horizon = 26

    window = 13
    seasonal = 52

    actual_values = np.array([value[0] for value in univariant_data.values[-forecast_horizon:]])
    train_data = np.array([value[0] for value in univariant_data.values[:-forecast_horizon]])

    model_predictions_df, model_inpredictions_df, model_result_df = get_model_predictions(model_dict, univariant_data, train_data, actual_values, forecast_horizon, seasonal)

    baseline_predictions_df, baseline_inpredictions_df, baseline_result_df = get_baseline_forecasts(baseline_model_dict, seasonal_baseline_model_dict, model_dict, univariant_data, train_data, actual_values, forecast_horizon, seasonal, window)

    all_results = pd.concat([model_result_df, baseline_result_df], axis=1).T.sort_values(by=['mpe'])

    all_predictions = pd.concat([baseline_predictions_df, model_predictions_df], axis=1)

    all_inpredictions = pd.concat([baseline_inpredictions_df, model_inpredictions_df], axis=1)

    model_1 = xgb.XGBRegressor()
    model_2 = lgb.LGBMRegressor()

    good_models = all_results.sort_values(by='rmse').index.tolist()

    model_1.fit(all_inpredictions[good_models], all_inpredictions['actual'])

    model_2.fit(all_inpredictions[good_models], all_inpredictions['actual'])

    testing_1 = pd.DataFrame(list(model_1.predict(all_predictions[good_models])),all_predictions['actual'].to_list()).reset_index()

    testing_2 = pd.DataFrame(list(model_2.predict(all_predictions[good_models])),all_predictions['actual'].to_list()).reset_index()

    xgboost_result = pd.DataFrame([get_metrics(testing_1, 0, 'index')])
    xgboost_result = xgboost_result.T.rename(columns={0:'xgboost'}).T

    lightgbm_result = pd.DataFrame([get_metrics(testing_2, 0, 'index')])
    lightgbm_result = lightgbm_result.T.rename(columns={0:'lightgbm'}).T

    all_results  = pd.concat([all_results, xgboost_result, lightgbm_result]).sort_values(by='rmse')

    final_results[cell] = all_results

    file_path = "dataframes.pkl"

    with open(file_path, "wb") as pkl_file:
        pickle.dump(final_results, pkl_file)


historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000074 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 740
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 12
[LightGBM] [Info] Start training from score 1904.808357
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

/var/folders/8y/wdvjjlbj6cnbpdh247s4gjw00000gs/T/ipykernel_92546/598552380.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univariant_data[single_column] = univariant_data[single_column].astype(float)


auto_ets_model
dynamic_theta_model
simple_exponential_smoothing_optimized_model
seasonal_exponential_smoothing_optimized_model
holt_model
holt_winters_model
historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000285 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 877
[LightGBM] [Info] Number of data points in the train set: 228, number of used features: 12
[LightGBM] [Info] Start training from score 8987.168619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

/var/folders/8y/wdvjjlbj6cnbpdh247s4gjw00000gs/T/ipykernel_92546/598552380.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univariant_data[single_column] = univariant_data[single_column].astype(float)


auto_ces_model
auto_ets_model
dynamic_theta_model
simple_exponential_smoothing_optimized_model
seasonal_exponential_smoothing_optimized_model
holt_model
holt_winters_model
historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 743
[LightGBM] [Info] Number of data points in the train set: 197, number of used features: 12
[LightGBM] [Info] Start training from score 4630.442503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

theta_model
optimized_theta_model
auto_ces_model
auto_ets_model
dynamic_theta_model
simple_exponential_smoothing_optimized_model
seasonal_exponential_smoothing_optimized_model
holt_model
holt_winters_model
historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001075 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 876
[LightGBM] [Info] Number of data points in the train set: 228, number of used features: 12
[LightGBM] [Info] Start training from score 4964.718293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

/var/folders/8y/wdvjjlbj6cnbpdh247s4gjw00000gs/T/ipykernel_92546/598552380.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univariant_data[single_column] = univariant_data[single_column].astype(float)


theta_model
optimized_theta_model
auto_ces_model
auto_ets_model
dynamic_theta_model
simple_exponential_smoothing_optimized_model
seasonal_exponential_smoothing_optimized_model
holt_model
holt_winters_model
historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003107 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 873
[LightGBM] [Info] Number of data points in the train set: 228, number of used features: 12
[LightGBM] [Info] Start training from score 704.457006
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/var/folders/8y/wdvjjlbj6cnbpdh247s4gjw00000gs/T/ipykernel_92546/598552380.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univariant_data[single_column] = univariant_data[single_column].astype(float)


theta_model
optimized_theta_model
auto_ces_model
auto_ets_model
dynamic_theta_model
simple_exponential_smoothing_optimized_model
seasonal_exponential_smoothing_optimized_model
holt_model
holt_winters_model
historic_average_baseline
naive_baseline
random_walk_baseline
seasonal_naive_baseline


In [ ]:
train_data

In [ ]:
final_results['BR-SMPPHAB_OV-ONLINE']

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MAE
from neuralforecast.auto import (
    AutoRNN,
    AutoLSTM,
    AutoGRU,
    AutoTCN,
    AutoDeepAR,
    AutoDilatedRNN,
    AutoMLP,
    AutoNBEATS,
    AutoNBEATSx,
    AutoNHITS,
    AutoTFT,
    AutoVanillaTransformer,
    AutoInformer,
    AutoAutoformer,
    AutoFEDformer,
    AutoPatchTST,
    AutoTimesNet,
    AutoStemGNN,
    AutoHINT,
)

model_list = {
    'AutoRNN':AutoRNN,
    'AutoLSTM':AutoLSTM,
    'AutoGRU':AutoGRU,
    'AutoTCN':AutoTCN,
    # 'AutoDeepAR': AutoDeepAR,
    'AutoDilatedRNN':AutoDilatedRNN,
    'AutoMLP':AutoMLP,
    'AutoNBEATS':AutoNBEATS,
    'AutoNBEATSx':AutoNBEATSx,
    'AutoNHITS':AutoNHITS,
    'AutoTFT':AutoTFT,
    'AutoVanillaTransformer':AutoVanillaTransformer,
    'AutoInformer':AutoInformer,
    'AutoAutoformer':AutoAutoformer,
    'AutoFEDformer':AutoFEDformer,
    'AutoPatchTST':AutoPatchTST,
    'AutoTimesNet':AutoTimesNet,
    'AutoStemGNN':AutoStemGNN,
    'AutoHINT':AutoHINT,
}

In [ ]:
new_uni_data = univariant_data.reset_index().rename(columns={'date':'ds', single_column:'y'}).head(1800)
new_uni_data['unique_id'] = 1.0

In [ ]:
import matplotlib.pyplot as plt
from time import sleep

# Try different hyperparmeters to improve accuracy.
all_forecasts = {}
for model_name, model in model_list.items():
    print(str(model))
    try:
        models = [model(h=forecast_horizon, backend='ray', num_samples=10, loss=MAE(), config=dict(max_steps=100))]
        nf = NeuralForecast(models=models, freq='D')
        nf.fit(df=new_uni_data[:-forecast_horizon])
        Y_hat_df = nf.predict()
        Y_hat_df = Y_hat_df.reset_index()
        Y_hat_df['actual'] = new_uni_data[-forecast_horizon:]['y'].values
        Y_hat_df.set_index('ds').drop(columns='unique_id').plot()
        all_forecasts[model_name] = Y_hat_df
        plt.show()
    except Exception as e:
        print(e)
        print(f'Failed {model_name}')
        continue


In [ ]:
get_metrics(Y_hat_df, 'AutoTCN', 'actual')

In [ ]:
results = pd.concat([value for value in all_forecasts.values()], axis=1)
results = results[[col for col in results.columns if 'Auto' in col]+['actual']].T.drop_duplicates().T

In [ ]:
result_metrics = pd.DataFrame([get_metrics(results, col, 'actual') for col in results.columns if 'Auto' in col]).T

In [ ]:
result_metrics.columns = [col for col in results.columns if 'Auto' in col]

In [ ]:
all_forecasts['AutoLSTM'].set_index('ds').drop(columns='unique_id').plot()

In [ ]:
result_metrics

In [ ]:
univariant_data.pivot_table

In [ ]:
univariant_data

In [ ]:
actual_values

In [ ]:
final_results['BE-INKJET_PRINTING_DEVICES-ONLINE']